In [1]:
from KeywordSearch import loader, indexing

In [2]:
#loader.process_first_k_books(k=500)

In [3]:
all_tokens = loader.load_token_vocab(k=-1)

In [4]:
bow_index, indexed_books = indexing.build_full_index(k=-1)

In [ ]:
from pympler import asizeof

415971552 366531768


In [ ]:
asizeof.asizeof(bow_index)

412098408

In [5]:
book_path_template = loader.token_dir + "PG%d_tokens.txt"

def read_tokens(PG_id: int):
    with open(book_path_template %PG_id, encoding="UTF-8", errors="ignore") as f:
        return loader.stemmer.stemWords(f.read().splitlines())

In [6]:
from gensim.corpora import Dictionary
from gensim.models import TfidfModel
from gensim.similarities import MatrixSimilarity
import concurrent.futures

docs = []
failed_jobs = 0
complete_counter = 0
with concurrent.futures.ThreadPoolExecutor() as pool:
    jobs = {
        pool.submit(
            read_tokens, book_id)
            : book_id for book_id in indexed_books
        }
    
    for job in concurrent.futures.as_completed(jobs):
        book_id = jobs[job]
        try:
            docs.append(job.result())
        except Exception as e:
            # raise e
            failed_jobs.append(book_id)
        complete_counter += 1
        print(f"Finished fetching tokens in {complete_counter} books...", end="\r")
        
        print(f"\n{len(failed_jobs)}/{len(jobs)} token fetching jobs failed")

In [ ]:
dictionary = Dictionary(docs)
corpus = [dictionary.doc2bow(doc) for doc in docs]
del docs
tfidf = TfidfModel(corpus, smartirs='ntc')
tfidf_corpus = [tfidf[doc] for doc in corpus]
del corpus, tfidf
index = MatrixSimilarity(tfidf_corpus, num_features=len(dictionary))

In [4]:
# import pickle
# with open("all_tokens.pkl", 'rb') as f:
#     k, offset, _all_tokens = pickle.load(f)
# with open("all_tokens.pkl", "wb") as f:
#     pickle.dump((k, offset, tuple(_all_tokens)), f)

In [7]:
from pympler import asizeof
asizeof.asizeof(bow_index), asizeof.asizeof(index)

(415963552, 22646504)

In [9]:
415963552/1024/1024

396.6937561035156

In [6]:
415971552/1024/1024

396.7013854980469

In [ ]:

mem_size_lst = asizeof.asizeof(all_tokens)
mem_size_tup = asizeof.asizeof(tuple(all_tokens))
mem_size_lst/1024/1024, mem_size_tup

In [12]:
from math import log2
import numpy as np
log2(len(all_tokens))
x = np.array([1,2,3])
x.shape[0]

3

In [3]:
#loader.build_full_index()

Finished building index for 429 books...
0/429 failures while building index


In [4]:
import pickle

In [5]:
with open("index.pkl", "rb") as f:
    index = pickle.load(f)

In [6]:
from pympler import asizeof
mem_size = asizeof.asizeof(index)

In [7]:
mem_size/1024/1024

319.3350143432617

In [10]:
index[1]

{80: array([10243, 10403], dtype=uint16),
 79: array([25780], dtype=uint16),
 48: array([ 6734,  7987, 16029, 21729, 24418, 36271, 39482, 42737, 50177,
        50986, 53304,  7012, 12227, 40460, 40703, 40782, 45415, 65362,
         8579, 10451, 11685, 58603, 60905, 22953], dtype=uint16),
 87: array([ 9931, 11277, 12834, 20154, 26448, 29587, 42158, 45577, 49165,
        58229, 59212, 61784, 17947, 23567, 52845, 53082, 53149, 53555,
        53561, 58426, 14091, 23403, 25391, 26582, 13410, 15989, 43851],
       dtype=uint16),
 152: array([18960], dtype=uint16),
 180: array([10541, 11974, 13667, 21596, 28334, 31832, 45235, 48950, 52794,
        62321, 63351,   356, 23336, 29218, 60503, 60841, 60928, 61272,
        61284, 22356, 32306, 34389, 35683, 27026, 29878, 62687],
       dtype=uint16),
 200: array([  508,   526,   565,   573,  1249, 17165, 29297, 60475, 10958],
       dtype=uint16),
 247: array([65472], dtype=uint16),
 248: array([616], dtype=uint16),
 323: array([3870, 3871, 4012, 4

In [20]:
size = 0
print(len(index))
for i, token in enumerate(index):
    for arr in token.values():
        size += asizeof.asizeof(arr)
    print(i, end='\r')
size

137799


240188992

In [21]:
size/1024/1024

229.06207275390625